<a href="https://colab.research.google.com/github/genie0320/gpt_writer/blob/colab/main_langchain_RAG_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project setting

In [9]:
!pip install --quiet langchain langchain-openai
!pip install --quiet pypdf chromadb tiktoken
!pip install --quiet icecream

In [10]:
import icecream as ic

In [11]:
# Load Keys

from google.colab import userdata
openai_api_key = userdata.get('OPENAI_API_KEY')

# from dotenv import load_dotenv
# load_dotenv()
# api_key = os.getenv("OPENAI_API_KEY")

In [12]:
# Load Drives

# GDrive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [13]:
# ___ setting ___
LLM_MODEL = "gpt-3.5-turbo"
MAX = 50
TEMP = 1.5

EMBED_MODEL = ""
SPLIT = 500
OVERRAP = 50
# _______________

from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate

llm = OpenAI(api_key=openai_api_key)
chat_model = ChatOpenAI(api_key=openai_api_key)
embed_model = ''

In [14]:
print("* Got the key" if openai_api_key else "Something goes wrong")
print(f'* LLM model set : \n - model : {LLM_MODEL} \n - max token : {MAX} \n - temperature : {TEMP}')
print(f'* Embed model set : \n - model : {EMBED_MODEL} \n - max token : {MAX} \n - temperature : {TEMP}')

* Got the key
* LLM model set : 
 - model : gpt-3.5-turbo 
 - max token : 50 
 - temperature : 1.5
* Embed model set : 
 - model :  
 - max token : 50 
 - temperature : 1.5


# LLM setting

In [15]:
# # Comletion
# prompt = PromptTemplate.from_template( '{time} + \n\n text{name} +  {action}')
# user_input = 'Where shall we go today?'
# new_prompt = prompt.format(
#     time = "old",
#     name = "Genie",
#     action = user_input
# )
# res_llm = llm.invoke(new_prompt)

# # Chat
# template = "You are my new friend. We met {place} for {activity}."
# human_template = "{text}"
# chat_prompt = ChatPromptTemplate.from_messages([
#     ("system", template), # 값을 tuple로 전달.
#     ("human", human_template),
# ])

# prompt = chat_prompt.format_messages(place="in library", activity="being a study friend", text="Hey, Sweety!")
# res_chat = chat_model.invoke(prompt).content

In [16]:
# print(prompt)
# print(res_chat)

# RAG

In [19]:
## loader
# 특정 폴더 안의 모든 파일을 불러올 수 있도록 구성.
# from langchain_community.document_loaders import DirectoryLoader > 이건 다 불러올 수 있는건가..?
# from langchain_community.document_loaders import UnstructuredPDFLoader # 바꿀 것

from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("gdrive/MyDrive/unsu.pdf")
pages = loader.load_and_split()
pages[0]

File /resources/planning_tip.pdf type detected as .pdf
File /resources/unsu.pdf type detected as .pdf
page_content='기획서 작성의 치트키\n\n보기 쉽고 읽기 쉬운 기획서 만드는 방법\n\n최성호 컨설턴트\n\n워크샵 참여 인증 이벤트\n\n기획서 작성 치트키를 확인하고 나의 생각과 소감을 자유롭게 더해서 워크샵 참여를 인증해주세요.\n\n참여하는 모든 분들께 일잘러 아이템을 선물로 드립니다!\n\n업무 스킬 레벨업을 위한 어도비 워크샵\n\n1탄 일잘러의 기획법\n\n어도비 워크샵 | 일잘러의 기획법\n\n이벤트 참여 방법 안내\n\n❶\n\n최성호 기획자의 팁을 확인하고,\n\n본 강의 자료에서 가장 인상 깊었던 부분에 애크로뱃으로 소감을 달아주세요.\n\n애크로뱃 [주석] 도구 → [텍스트 강조]로 인상 깊은 부분에 형광펜 표시,\n\n스티커 노트 추가\n\n텍스트 강조\n\n[스티커 노트 추가]로 소감을 남겨주세요.\n\n❷\n\n공유 링크로 저장해서 제출하면 응모 끝!\n\n우측 상단 [링크 아이콘] 클릭 → [링크 복사] → 참여 페이지에 해당 링크를 입력하세요.\n\n※ 안내 방안에 따라 생성된 링크가 아닌 다른 링크로 입력하는 경우,\n\n이벤트 참여가 취소됩니다.\n\nTIP) 아직 애크로뱃이 없다면?\n\n7일 무료 체험으로 경험하세요!\n\n앱 설치 없이 웹에서 간편하게!\n\nPDF 완벽 솔루션 애크로뱃의 모든 기능을 무료로 사용해 보세요.\n\n애크로뱃 구매 전이라도 다양한 기능을 웹에서 바로 체험해볼 수 있습니다.\n\n애크로뱃 무료 체험 하기\n\n온라인 서비스 사용하기\n\n출처 : 누구나 탐내는 실전 기획서, 최성호 저\n\n기획서 작성의 치트키 목차\n\n❶\n\n기획서의 본질 이해하기\n\n❷\n\n기획자 마인드 만들기\n\n❸\n\n기획서의 흐름 이해하기\n\n❹\n\n보기 쉬운 기획서 만들기\n\n❺\n\n읽기 쉬운 기획서 만들기\n

In [20]:
## Transform (chunking)
# with open("../../state_of_the_union.txt") as f: # 이건 로컬파일일 때...이야기인가. loader와 무슨 관계?
#     state_of_the_union = f.read()

# from langchain_experimental.text_splitter import SemanticChunker > 이건 openai의 실험적 기능이라는데... 한번 써보고 싶음.
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    # Set a really small chunk size, just to show.
    chunk_size=300,
    chunk_overlap=30,
    # length_function=len, > tiktoken 사용시에는 왠지... 비활성해야 함.
    is_separator_regex=False,
)

# texts = text_splitter.create_documents([state_of_the_union])
texts = text_splitter.split_documents(pages)
# print(texts[0])
print(texts[0])
print(texts[1])

page_content='기획서 작성의 치트키\n\n보기 쉽고 읽기 쉬운 기획서 만드는 방법\n\n최성호 컨설턴트\n\n워크샵 참여 인증 이벤트\n\n기획서 작성 치트키를 확인하고 나의 생각과 소감을 자유롭게 더해서 워크샵 참여를 인증해주세요.' metadata={'source': '/tmp/tmpzdapti7a/tmp.pdf'}
page_content='참여하는 모든 분들께 일잘러 아이템을 선물로 드립니다!\n\n업무 스킬 레벨업을 위한 어도비 워크샵\n\n1탄 일잘러의 기획법\n\n어도비 워크샵 | 일잘러의 기획법\n\n이벤트 참여 방법 안내\n\n❶\n\n최성호 기획자의 팁을 확인하고,' metadata={'source': '/tmp/tmpzdapti7a/tmp.pdf'}


In [21]:
# Embedding & Store to vectorDB
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

# chroma_client = chromadb.Client() > 이건 뭐에 필요한 것인지...
embeddings_model = OpenAIEmbeddings(api_key=openai_api_key)

db = Chroma.from_documents(texts, embeddings_model)

user_input_embedding = input('검색할 키워드 입력 :')
query = user_input_embedding
docs = db.similarity_search(query)
print(docs[0].page_content)

검색할 키워드 입력 :기획이란
어도비 워크샵 | 일잘러의 기획법

1. 기획서의 본질 이해하기

‘기획’이란?

내 주장을 관철 시키기 위해 상대방을 설득하는 과정 ‘장난감을 사 달라고 떼 쓰는 아이를 달래기’,


In [22]:
## Store
# Done with embedding

In [23]:
## Retrieve from DB
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI

# user_input_multiquery = input('검색할 키워드 입력 :')
# question = user_input_multiquery
# llm = chat_model

# retriever_from_llm = MultiQueryRetriever.from_llm(
#     retriever=db.as_retriever(), llm=llm
# )
# print(retriever_from_llm)
# print(len(res))
# res = retriever_from_llm.get_relevant_documents(query = question)

In [26]:
## Make answer with LLM
from langchain.chains import RetrievalQA

user_input_multiquery = input('검색할 키워드 입력 :')
question = user_input_multiquery
llm = chat_model
qa_chain = RetrievalQA.from_chain_type(llm, retriever=db.as_retriever())
result = qa_chain.invoke({'query':question})
# print(result)
print(result['result'])

검색할 키워드 입력 :기획이란
기획은 내 주장을 관철시키기 위해 상대방을 설득하는 과정을 말합니다. 예를 들어, '장난감을 사 달라고 떼쓰는 아이를 달래기'나 '고객에게 우리 제품을 판매하기'와 같이 우리가 이미 살면서 수많은 기획을 경험하고 있습니다. 기획은 설득의 과정을 담아낸 문서인 기획서를 작성하는 것도 포함됩니다. 기획서는 설득의 과정을 담아낸 문서이므로 핵심만 정리하여 짧은 시간에 설득할 수 있도록 작성해야 합니다.


# Issue

## format_instructions
prompt의 Output 의 형태를 정해줄 수도 있다. `\n\n{format_instructions}`  
아마도 몇개를 보여줘라... 할 때 어떻게 보여줄지를 알려주는 대목인 모양.
그런데 이건 output parser 에서도 정해줄 수 있고 물론 prompt자체에서 해결할 수도 있다. 왜 여기서 굳이?

In [25]:
template = "Generate a list of 5 {text}.\n\n{format_instructions}" # \n\n 는 왜 필요한거지?

chat_prompt = ChatPromptTemplate.from_template(template)
chat_prompt = chat_prompt.partial(format_instructions=output_parser.get_format_instructions()) # format_instructions에 따라서 출력해라 ? 굳이 왜 이렇게 하는지 모르겠음.
chain = chat_prompt | chat_model | output_parser
chain.invoke({"text": "human_name"}) # 위의 teamplate > text 에 human_name을 넣어서 대답을 가져와라.

NameError: name 'output_parser' is not defined

## schema
● 일단 포기.
schema와 template가 뭐가 다른 건지 모르겠음.  
> 둘다 일종의 foramtting을 위한 것인데...


문서에 따르자면, 순수하게 개발경험향상을 위해서 추가한 것이라고 한다. 하지만 다음과 같이 권고하고 있다.

- 단순한 메시지 전달을 위해서는 `Message`를 쓰고,
- 뭔가 변수를 통해 전달해야 할 것이 있는 경우에는 `MessageTemplate`을 활용하라고 한다.

In [ ]:
from langchain.schema import AIMessage, HumanMessage, SystemMessage
# llm.invoke("hi?")

# 단순한 프롬프트
# message = "Hi?"
# llm.invoke(message)

# # 복잡한 프롬프트
# template = 'You are a {time} friend of mine'
# human_template = 'And I want you to {activity}'
# ai_answered = 'Sure! why not!'

# prompt = (
#     SystemMessage(content = template) + HumanMessage(content = human_template) + AIMessage(content = ai_answered)
# )

# message01 = prompt.format_messages(
#     time = "old",
#     activity = "help my love life",
# )

# message02 = prompt.format_prompt(
#     time = "old",
#     activity = "help my love life",
# )

# prompt02 = SystemMessage(content="You are a nice pirate")
# prompt02_ = (
#     prompt02 + HumanMessage(content="hi") + AIMessage(content="what?") + "{input}"
# )
# message03 = prompt02_.format_messages(input="i said hi")
# res = new_prompt.format_messages(input="i said hi")

# print(f'format_messages_prompt : {message01}')
# print(f'answer : {chat_model.invoke(message01)}')
# print('********************************************')
# print(f'format_prompt_prompt : {message02}')
# print(f'answer : {chat_model.invoke(message02)}')
# print('********************************************')
# print(f'given : {message03}')
# print(f'answer : {chat_model.invoke(message03)}')

# print(f'res : {res}')

In [ ]:
from langchain.chains import LLMChain
# from langchain_openai import ChatOpenAI

model = ChatOpenAI(api_key=openai_api_key)
chain = LLMChain(llm=model, prompt=prompt)
chain.run(topic="sports", language="korean")

--------------------------------------------------------------------------------

# Understanding output parser

In [ ]:
from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
output_parser.parse("hi, bye")
# >> ['hi', 'bye']

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are new friend of mine"),
    ("user", "{user_input}")
])

In [ ]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

In [ ]:
chain = prompt | llm | output_parser

In [ ]:
ai_answered = chain.invoke({"user_input": "Hi, there!"})
print(f'New friend from LLMs : {ai_answered}')

# Few-shot prompt templates

Page link found | Content | As is link | to be link|
------------------|-----------|---------|-----------|
https://python.langchain.com/docs/modules/model_io/prompts | How to use few-shot examples with LLMs | https://python.langchain.com/docs/modules/model_io/few_shot_examples |https://python.langchain.com/docs/modules/model_io/prompts/few_shot_examples


In [ ]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "Tell me a short {adjective} joke about {content}."
)
prompt_template.format(adjective="funny", content="love")

In [ ]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

In [ ]:
examples = [
    {
        "question": "Hi, honey?",
        "answer": """
Yes,
my love.
When did you get here?
""",
    },
    {
        "question": "We just arrived. What should we eat?",
        "answer": """
Now I'm very hungry, so I think I can eat anything.
""",
    },
    {
        "question": "Honey, What is your favorite animation these days?",
        "answer": """
You know I like "Love Live", right? A new season is on air these days.
""",
    },
    {
        "question": "Really? Then do you want to go see it together?",
        "answer": """
No, it's okay. You're busy, so let's wait until the next movie comes out and watch it together.
""",
    },
]

example_prompt = PromptTemplate(
    input_variables=["question", "answer"], template="Question: {question}\n{answer}"
)

print(example_prompt.format(**examples[0]))

# For wordwrap for out cell.

In [ ]:
# # from ipywidgets import Textarea, widgets
# # from IPython.display import display

# output_to_display = res.content

# # print(~) > res = ~     >> res.contents 등 가공. output_to_display >>
# # 다시 위쪽의 코드에 반영.

# output_widget = Textarea(
#     value=output_to_display,
#     layout=widgets.Layout(width="100%", height='300px')  # Can change Height
# )
# display(output_widget)

function KeepClicking(){
console.log("Clicking");
document.querySelector("colab-connect-button").click()
}
setInterval(KeepClicking,60000)